In [1]:
# импортируем все необходимые библиотеки
import psycopg2
import getpass
import numpy as np
from datetime import datetime
from datetime import time
import pandas as pd
import dateutil.relativedelta as dr

user=input('User of database:')
database=input('Name of database:')
host=input('Name of host')
port=input('Name of port')



# соединяемся со слоном
conn=psycopg2.connect(user=user, 
                      database=database, 
                      host=host,
                      port=port,
                      password=getpass.getpass('Insert password'))
# если что-то есть, то работает
print(conn.get_dsn_parameters())

User of database:postgres
Name of database:postgres
Name of hostlocalhost
Name of port5432
Insert password········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [60]:
#импортируем  данные
raw_data="""SELECT "ISINCode" as "ISIN"
                        ,"FaceValue" 
                        ,"HaveOffer"
                        ,"AmortisedMty"
                        ,"CouponPerYear"
                        ,"NumCoupons"
                        ,"EndMtyDate"
                        , "CouponType"
                        ,"IssuerName"
                  FROM bond_description_task  
                  WHERE "Status"!='Погашен'"""
# max CPN нужен для определения изначального размера купона по амортизируемым бумагам
raw_data=pd.read_sql_query(raw_data, conn) 
additional_data="""SELECT MAX("CPN") as "CPN"
                  , "ISIN"
                  ,max("BUYBACKDATE") as "BUYBACKDATE"
                  ,max("BUYBACKPRICE") as "BUYBACKPRICE"
                  FROM quotes_task 
                  group by "ISIN" """
additional_data=pd.read_sql_query(additional_data, conn) 
#сохраняем необходимые данные для работы в формате dataframe
df = additional_data.merge(raw_data, how='inner', on='ISIN')
# удаляем все битые данные, исходя из которых нельзя провести рассчеты (например, если есть число купонов, но нет их размера)
df.BUYBACKPRICE = df.BUYBACKPRICE.fillna(0)
df = df[~df.CPN.isna()].reset_index(drop = True)
df = df[df.CPN != 0]
df = df[df.CouponPerYear > 0]

# Если у вас есть timestamp format, лучше изменить на data, так как это может вызвать проблемы с кодом
# Пример для buybackdate
# data.BUYBACKDATE = data.BUYBACKDATE.dt.date

In [61]:
def coupon_count(coup_str):
    buf = pd.DataFrame()
    end_date = coup_str.BUYBACKDATE.values[0] if coup_str.BUYBACKDATE.values[0] is not None else coup_str.EndMtyDate.values[0]
    a = int(coup_str.CouponPerYear.values[0])
    b = float(coup_str.NumCoupons.values[0])
    types = True
    if coup_str.CPN is not None or coup_str.CPN != 0:
        if coup_str.AmortisedMty.astype(str).values[0] == 'False':
            types = False
            amount = coup_str.CPN.values[0]
            for j in np.linspace((b-1)/a, 0, b):
                date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25*j))
                if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                    date = end_date
                    amount = coup_str.BUYBACKPRICE.values[0]
                    buf = pd.concat([buf, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                    date = end_date
                    amount += coup_str.FaceValue.values[0]
                    buf = pd.concat([buf, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                buf = pd.concat([buf, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                amount = coup_str.CPN.values[0]
        else:
            percent = float(coup_str.CPN.values[0]) / float(coup_str.FaceValue.values[0])
            index = (b-1)
            face = coup_str.FaceValue.values[0]
            while index >= 0:
                amount = percent * face
                date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25*index/a))
                if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                    date = end_date
                    amount = coup_str.BUYBACKPRICE.values[0]
                    buf = pd.concat([buf, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                    date = end_date
                    amount = face
                    buf = pd.concat([buf, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                buf = pd.concat([buf, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                index -= 1
                face = face * (1 - percent)
    my_index = pd.MultiIndex.from_tuples([(coup_str.ISIN.values[0], 'Amortised' if types else 'Not Amortised'
                                           , coup_str.IssuerName.values[0] )], names=['ISIN', 'Amortisation', 'Issuer'])
    buf = pd.DataFrame(buf.values, index = my_index, columns = buf.columns)
    return buf

In [62]:
#buf - итоговая таблица для всех эмитентов и дат
buf = pd.DataFrame()
for i in df.iterrows():
    buf = pd.concat([buf, coupon_count(pd.DataFrame(i[1]).T)], sort = True)
buf.fillna(0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  # This is added back by InteractiveShellApp.init_path()


,,,2006-08-19,2006-10-06,2007-01-05,2007-02-18,2007-04-06,2007-06-11,2007-06-15,2007-06-16,2007-07-07,2007-08-11,...,2031-03-21,2031-09-20,2032-03-21,2032-09-19,2033-03-21,2033-09-20,2034-03-21,2034-09-20,2035-03-21,2035-09-20
ISIN,Amortisation,Issuer,,,,,,,,,,,,,,,,,,,,,
RU000A0GN9B5,Not Amortised,Мосэнерго (ТГК-3),41.14,0.0000,0.0000,41.14,0.0000,0.00,0.00,0.00,0.0000,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JNGU3,Not Amortised,Банк ВТБ,0.00,30.2918,30.2918,0.00,30.2918,0.00,0.00,0.00,30.2918,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JNYN1,Not Amortised,Москва Правительство,0.00,0.0000,0.0000,0.00,0.0000,40.11,0.00,0.00,0.0000,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JNYP6,Not Amortised,Москва Правительство,0.00,0.0000,0.0000,0.00,0.0000,0.00,40.11,0.00,0.0000,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JNZW9,Not Amortised,Вагонмаш,0.00,0.0000,0.0000,0.00,0.0000,0.00,0.00,77.29,0.0000,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JP2G4,Not Amortised,Россельхозбанк,0.00,0.0000,0.0000,0.00,0.0000,0.00,0.00,0.00,0.0000,46.12,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JPF47,Not Amortised,Россельхозбанк,0.00,0.0000,0.0000,0.00,0.0000,0.00,0.00,0.00,0.0000,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JPK81,Not Amortised,МеждународПромБанк,0.00,0.0000,0.0000,0.00,0.0000,0.00,0.00,0.00,0.0000,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JPMT8,Not Amortised,Россельхозбанк,0.00,0.0000,0.0000,0.00,0.0000,0.00,0.00,0.00,0.0000,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [63]:
#от какой-то даты до конца таблицы
def my_fun_end(issuer_name,time):
    Buffercolumns=pd.DataFrame()
    for i in range(0,len(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns)):
        date_size=int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[0])*365+int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[1])*365/12+int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[2])
        if int(time.split('-')[0])*365+int(time.split('-')[1])*365/12+int(time.split('-')[2])<date_size:
            dfd=buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).iloc[:, i:i+1]
            Buffercolumns=pd.concat([Buffercolumns,dfd], axis = 1)
    return Buffercolumns

In [64]:
# Дату вводить только в формате 'YYYY-MM-DD'
my_fun_end('РЖД','2020-01-01')

,,2020-03-21,2020-03-31,2020-05-03,2020-09-19,2020-11-01,2020-11-13,2021-03-21,2021-09-19,2022-03-21,2022-09-20,...,2031-03-21,2031-09-20,2032-03-21,2032-09-19,2033-03-21,2033-09-20,2034-03-21,2034-09-20,2035-03-21,2035-09-20
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JQ5Q5,Not Amortised,0.0000,0.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JQ656,Not Amortised,0.0000,0.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JQ6S9,Not Amortised,0.0000,0.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JQ7W9,Not Amortised,0.0000,0.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JQ7X7,Not Amortised,0.0000,0.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JQ7Z2,Not Amortised,0.0000,0.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JQRD9,Not Amortised,0.0000,0.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JSGV0,Not Amortised,0.0000,0.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JTU85,Not Amortised,40.8900,100.0,0.00,0.0000,0.00,0.0,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [65]:
#на дату
def my_fun_date(issuer_name,time):
    Buffercolumns=pd.DataFrame()
    for i in range(0,len(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns)):
        date_size=int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[0])*365+int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[1])*365/12+int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[2])
        if int(time.split('-')[0])*365+int(time.split('-')[1])*365/12+int(time.split('-')[2])==date_size:
            dfd=buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).iloc[:, i:i+1]
            Buffercolumns=pd.concat([Buffercolumns,dfd], axis = 1)
    return Buffercolumns

In [66]:
# Дату вводить только в формате 'YYYY-MM-DD'
my_fun_date('РЖД','2035-09-20')

,,2035-09-20
ISIN,Amortisation,
RU000A0JQ5Q5,Not Amortised,0.0000
RU000A0JQ656,Not Amortised,0.0000
RU000A0JQ6S9,Not Amortised,0.0000
RU000A0JQ7W9,Not Amortised,0.0000
RU000A0JQ7X7,Not Amortised,0.0000
RU000A0JQ7Z2,Not Amortised,0.0000
RU000A0JQRD9,Not Amortised,0.0000
RU000A0JSGV0,Not Amortised,0.0000
RU000A0JTU85,Not Amortised,0.0000


In [67]:
#от какой-то начала таблицы до сегодня
acctual_date=datetime.now().strftime("%Y-%m-%d")
def my_fun_today(issuer_name):
    Buffercolumns=pd.DataFrame()
    for i in range(0,len(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns)):
        date_size=int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[0])*365+int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[1])*365/12+int(buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).columns[i].split('-')[2])
        if int(datetime.now().strftime("%Y-%m-%d").split('-')[0])*365+int(datetime.now().strftime("%Y-%m-%d").split('-')[1])*365/12+int(datetime.now().strftime("%Y-%m-%d").split('-')[2])>date_size:
            dfd=buf.xs(str(issuer_name), level=2).dropna(axis=1, how='all').fillna(0).iloc[:, i:i+1]
            Buffercolumns=pd.concat([Buffercolumns,dfd], axis = 1)
    return Buffercolumns

In [68]:
# Дата не нужна
my_fun_today('РЖД')

,,2009-11-14,2009-12-05,2009-12-20,2010-01-07,2010-01-13,2010-01-14,2010-05-16,2010-06-05,2010-06-20,2010-07-08,...,2018-07-18,2018-07-26,2018-09-20,2018-11-02,2019-03-21,2019-03-22,2019-05-03,2019-09-20,2019-11-02,2019-11-15
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JQ5Q5,Not Amortised,74.3,0.00,0.00,0.00,0.00,0.0000,74.3,0.00,0.00,0.00,...,0.00,0.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0
RU000A0JQ656,Not Amortised,0.0,71.45,0.00,0.00,0.00,0.0000,0.0,71.45,0.00,0.00,...,0.00,0.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0
RU000A0JQ6S9,Not Amortised,0.0,0.00,55.65,0.00,0.00,0.0000,0.0,0.00,55.65,0.00,...,0.00,0.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0
RU000A0JQ7W9,Not Amortised,0.0,0.00,0.00,0.00,0.00,57.8411,0.0,0.00,0.00,0.00,...,0.00,0.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0
RU000A0JQ7X7,Not Amortised,0.0,0.00,0.00,0.00,57.34,0.0000,0.0,0.00,0.00,0.00,...,0.00,0.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0
RU000A0JQ7Z2,Not Amortised,0.0,0.00,0.00,67.32,0.00,0.0000,0.0,0.00,0.00,67.32,...,0.00,0.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0
RU000A0JQRD9,Not Amortised,0.0,0.00,0.00,0.00,0.00,0.0000,0.0,0.00,0.00,0.00,...,59.84,100.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0
RU000A0JSGV0,Not Amortised,0.0,0.00,0.00,0.00,0.00,0.0000,0.0,0.00,0.00,0.00,...,0.00,0.0,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0
RU000A0JTU85,Not Amortised,0.0,0.00,0.00,0.00,0.00,0.0000,0.0,0.00,0.00,0.00,...,0.00,0.0,40.8900,0.0000,0.0000,40.89,0.0000,40.8900,0.0000,0.0


In [ ]:
# Фидбэк:
# 1.Соединить Python c БД по API. + 1 балл
# 2.Из Python запросить информацию, необходимую для пункта 3. +2 балла
# 3.Написать функцию, которая возвращает матрицу платежей облигаций заданного эмитента на заданную дату. Аргументы: код эмитента, дата матрицы платежей. Привести иллюстрационный пример использования. Привести иллюстрационный пример использования. 
#  Странно, что у неамартизируемой облигации (например, RU000A0JV8Q2) поток не оканчивается погашением. И еще я не понял, как учтены оферты.
# +2 балл.
# 4. Предусмотреть исполнение SQL запроса из п.2 внутри функции из п. 3. Запрос должен храниться в выходных данных функции. 
# Запихайте запрос в функцию и получите балл. 0 баллов.
# Текущая сумма: 5 балла.